In [21]:
import numpy as np
import pandas as pd
import torch
import torch_geometric
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data, Batch
from torch_geometric.transforms import LocalDegreeProfile
import torch.nn.functional as F
from torch_geometric.datasets import GemsecDeezer
from sklearn.metrics import accuracy_score, roc_auc_score

In [2]:
dataset = GemsecDeezer(root='.', name='HU')
data = dataset[0]
print(dataset)
print(data)
print()
print('num_nodes:', data.num_nodes)
print('num_edges:', data.num_edges)
print('is_directed:', data.is_directed())
print('has_self_loops:', data.has_self_loops())
# Automatically creating 5 node features
ldp = LocalDegreeProfile()
data = ldp(data)
data

GemsecDeezer()
Data(edge_index=[2, 445774], y=[47538, 84])

num_nodes: 47538
num_edges: 445774


/home/sina/anaconda3/envs/pyg/lib/python3.8/site-packages/torch_geometric/data/storage.py:259: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set '{'edge_index', 'y'}'. Please explicitly set 'num_nodes' as an attribute of 'data' to suppress this warning
  warnings.warn(


is_directed: False
has_self_loops: False


Data(edge_index=[2, 445774], y=[47538, 84], x=[47538, 5])

In [3]:
class GNN(torch.nn.Module):
    def __init__(self, in_channels, hid_channels, out_channels):
        super(GNN, self).__init__()
        torch.manual_seed(0)
        self.conv1 = GCNConv(in_channels=in_channels, out_channels=hid_channels)
        self.conv2 = GCNConv(in_channels=hid_channels, out_channels=hid_channels)
        self.conv3 = GCNConv(in_channels=hid_channels, out_channels=out_channels)
        # self.lin1 = torch.nn.Linear(in_features=hid_channels, out_features=out_channels)

    def forward(self, x, edge_index, index=0):
        # print(x)
        x = self.conv1(x, edge_index)
        # print(x)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv3(x, edge_index)
        # x = self.lin1(x)
        # x = x.sigmoid()
        return x
    

In [29]:
in_channels = data.num_features
hid_channels = 128
out_channels = data.y.shape[1]

model = GNN(in_channels=in_channels, hid_channels=hid_channels, out_channels=out_channels)
print(model)
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)


def train():
    model.train()
    optimizer.zero_grad()  # Clear gradients.
    out = model(data.x, data.edge_index)  # Perform a single forward pass.
    loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss solely based on the training nodes.
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.
    return loss

def test():
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)  # Use the class with highest probability.
    test_correct = pred[data.test_mask] == data.y[data.test_mask]  # Check against ground-truth labels.
    test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  # Derive ratio of correct predictions.
    return test_acc

def device_finder():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    return device


# for epoch in range(0, 11):
#     loss = train()
#     if epoch % 10 == 0:
#         print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
losses = []

for epoch in range(5):
    output = model(data.x, data.edge_index)
    loss = criterion(output, data.y.float())
    losses.append(loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    temp_outputs = output.clone().detach().numpy()
    temp_labels = np.array(data.y)
    score = roc_auc_score(temp_labels, temp_outputs)
    loss = float("{:.3f}".format(loss.item()))
    score = float("{:.3f}".format(score))
    print(f'Loss: {loss}, AUC Score: {score}')

GNN(
  (conv1): GCNConv(5, 128)
  (conv2): GCNConv(128, 128)
  (conv3): GCNConv(128, 84)
)
Loss: 1.287, Score: 0.499
Loss: 0.417, Score: 0.539
Loss: 0.258, Score: 0.551
Loss: 0.227, Score: 0.556
Loss: 0.218, Score: 0.558
